## word2vec 속도 개선

앞서 구현한 SimpleCBOW에서 입력층 -> 은닉층으로 MatMul이 사용되고    
은닉층 -> SoftmaxWithLoss계층으로 softmax손실값이 계산되었다    
   
그런데 이 때 **어휘의 개수가 늘어날 경우**와 **뉴런의 개수가 많아질 경우** 엄청난 연산능력이 필요   
-> 따라서 효율적으로 학습할 수 있는 방법을 찾는 것이 필요

### 1. 입력층 -> 은닉층 계산연산 효율화 하기 (Embedding 계층구현)

- 우리에게 입력되는 데이터 형태: one-hot encoding 된 형태 \[\[0, 0, 0, 1\]\]
- 여기에 어떤 W를 곱하는 행위는 결국 W에서 입력층의 1에 해당하는 값을 선정하는 작업임
    -> 곱셈을 수행하지 않고 특정 행의 데이터만 추출(slicing)하는 방법으로 변경

In [1]:
# Embedding 계층 구현
import numpy as np

W = np.arange(21).reshape(7,3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [3]:
W[2]

array([6, 7, 8])

In [4]:
#인덱스에 행렬을 넣어서 원하는 값들을 추출하기

idx = np.array([1, 0, 3, 0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

### Embedding backward 계층에 대한 설명

https://www.youtube.com/watch?v=OHgpvzPcxK0&t=5s&ab_channel=KyungHoonHan

- 곱셈노드의 역전파는 이전 계층 grads * 반대편 노드의 T
    - ex) W의 역전파 값은 $\frac{\partial w}{\partial h} * x^T$
    - x에 동일한 idx들이 n번 출몰할 경우
    - 역전파 계산과정에서 $\frac{\partial w}{\partial h} * x^T$에 의해 n번 더해지는 꼴이 나온다 (강의 19분 부터)
    - 따라서 처음 등장한 idx에 동일한 값을 덮어씌우는 것이 아닌 **더하는 작업**을 수행하도록 계산을 해야한다
    
- for구문을 써서 더하는 방식보다 np.add.at(A,idx,B)방식을 사용하는 것이 효율적임
    - ex) 만약 idx가 \[0, 0, 2, 1\] 이라면 
    - A[0] + B[0], A[0] + B[1], A[2] + B[2], A[1] + B[3] 작업이 수행되어 A의 값이 변한다

### 2. 은닉층 -> 손실층으로의 계산 효율화 (네거티브 샘플링)
- 네거티브 샘플링 Negative Sampling: 다중 클래스 분류문제를 이진 분류 문제로 근사하여 해결하는 방식

1) SimpleCBOW에서 다중 클래스 분류   
- 출력층은 단어의 개수에 해당하는 n벡터가 생성되고, 이 벡터를 이용하여 SoftMax층에 보내면 n개의 값들에 대한 각각의 확률값을 변환한다
    - 즉 n개의 확률값들이 산출된다
- 그리고 Loss층에 확률값들을 보내면 실제 정답데이터와 비교하여 손실을 반환한다

2.) CBOW에서 이진 분류 문제   
2.1) Embedding Dot: h * W[:, t]를 수행하여 슬라이싱 연산을 수행하는 층 
- 정답데이터는 입력의 윈도우들 가운데 있는 어떤 한 단어이다
- 오직 이 단어가 맞는지 아닌지에 대한 문제, 즉 **참 / 거짓 문제**로 치환하여 이 문제에 대한 확률값을 반환하도록 한다
    - 이 때 참 / 거짓 문제를 구현하기 위해서 정답 데이터 t에 해당하는 열의 idx만 추출하여 W와 계산을 수행하면 된다
- 이 때 Sigmoid함수를 사용하여 확률값을 산출한다

2.2) Sigmoid with Loss 계층 구현
- 최종 역전파 값은 y-t 즉 예측값 - 실제값    

<br >

**Negative Sampling**
- 단순히 정답 데이터(긍정적 예) 하나만을 가지고 손실을 구하는 것이 아니라
- 정답 외의 오답 데이터(부정적 예) 일부(샘플)도 추가하여서 손실을 구하는데 추가
- 즉 총 손실은 정답 데이터에 대한 손실 + 오답 데이터에 대한 손실을 구하는 것
    - 따라서 손실이 최소화 되려면 정답에 대한 확률이 커져야하고 오답에 대한 확률이 작아져야 한다
- 오답 데이터를 선정함에 있어서 corpus의 단어 출현 빈도를 바탕으로 선정하는 것이 좋다
    - 즉 corpus에서 자주 등장하는 단어를 오답 데이터에 선정하여 네거티브 샘플링을 수행하는것이 좋다
    - 왜냐하면 빈번하게 등장하는 단어를 이용하여 학습된 모델이 현실의 문제에서도 잘 해결할 가능성이 높기 때문

In [6]:
# Embedding Dot 클래스 구현
from mh_common.mh_layers import Embedding

class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis = 1) #내적을 수행해야 하므로 행렬곱이 아닌 원소별 곱을 한 후 행합을 수행
        
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtraget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

In [8]:
#corpus 등장 단어 확률분포에 따른 네거티브 샘플링 예시

np.random.choice(10) #랜덤하게 어떤 수를 뽑는 메서드 

9

In [10]:
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words)

'you'

In [12]:
np.random.choice(words, size = 5) #일정 수를 뽑도록 size를 설정

array(['say', 'I', 'I', 'say', 'say'], dtype='<U7')

In [13]:
np.random.choice(words, size = 5, replace = False) #replace로 복원, 비복원 추출 여부 결정

array(['you', 'say', '.', 'I', 'hello'], dtype='<U7')

In [20]:
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p = p) #확률에 따라서 추출

'you'

수정된 식 사용

### $\hat{P}(w_i) = \frac{p(w_i)^{0.75}}{\sum_{j}^{n}P(w_i)^{0.75}} $
- 기존 확률에 모두 0.75 제곱을 씌워준 형태로 변형
- 0에 가까운 확률을 좀 더 크게 보정하기 위해 사용함

In [21]:
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)
new_p #0.29, 0.01의 값이 커지고 0.7의 값은 작아졌다

array([0.64196878, 0.33150408, 0.02652714])

In [31]:
#UnigramSampler 구현
import sys
sys.path.append(r'C:\Users\myunghoon_k\OneDrive - 서울시립대학교\bitamin\dl_nlp_study\deep-learning-from-scratch-2-master')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss

import collections

class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None
        
        counts = collections.Counter() #단어들의 출현 빈도를 담은 객체
        for word_id in corpus:
            counts[word_id] += 1
            
        vocab_size = len(counts)
        self.vocab_size = vocab_size
        
        self.word_p = np.zeros(vocab_size) #단어 출현확률을 담은 객체
        for i in range(vocab_size):
            self.word_p[i] = counts[i]
            
        self.word_p = np.power(self.word_p, power) #보정된 출현확률 생성
        self.word_p /= np.sum(self.word_p)
        
    def get_negative_sample(self, target):
        batch_size = target.shape[0] #미니배치 방식을 이용해 학습하므로 n개 배치에 대한 n개의 정답 데이터가 존재할 것
        
        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype = np.int32)
            
            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0 #정답에 해당하는 확률을 0으로 만들어서 정답데이터를 뽑지 않도록 설정
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size = self.sample_size, replace = False, p = p)
                #비복원추출 방식을 사용하고 p확률에 따라서 전체 vocab_size길이 중 negative_sample_size만큼 랜덤하게 추출
        else:
            negative_sample = np.random.choice(self.vocab_size, size = (batch_size, self.sample_size),
                                              replace = True, p = self.word_p)
            
        return negative_sample

In [32]:
corpus = np.array([0,1,2,3,4,1,2,3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample) #정답외에 다른 오답 데이터들이 2개씩 배치마다 추출되는 것을 알 수 있음

[[4 3]
 [1 2]
 [2 4]]


In [35]:
#네거티브 샘플링 구현
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power = 0.75, sample_size = 5):
        self.sample_size = sample_size #배치 사이즈 선정
        self.sampler = UnigramSampler(corpus, power, smaple_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)] 
        #정답 데이터를 다룰 계층1, 오답 데이터를 다룰 계층 sample_size이므로 총 계층의 수는 sample_size + 1이 되어야 함
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
        
        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)
        
        #정답 데이터 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype = np.int32) #정답데이터들에 대한 손실을 계산하기 위한 기준값 1
        loss = self.loss_layers[0].forward(score, correct_label)
        
        #오답 데이터 순전파
        negative_label = np.zeros(batch_size, dtype = np.int32) #오답데이터들에 대한 손실을 계산하기 위한 기준값 0 (오답은 곧 0이 정답이므로)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)
            
        return loss
    
    def backward(self, dout = 1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout) #손실계층의 역전파 값
            dh += l1.backward(dscore) #
            
        return dh